# Part-of-speech Tagger con Keras

In [4]:
import keras
from keras.preprocessing import text

import numpy as np
import pickle
import pandas as pd

from collections import Counter

In [22]:
# Apriamo il file del postagger con pandas
data = pd.read_table("PoSTaggingCorpus/devel", sep="\s*")
#visualizziamo il risultato
#print textfile.words
#texfile.tags
words = list(set(data.words))
n_words = len(words)
tags = list(set(data.tags))
n_tags = len(tags)

print "#words:",n_words,"#tags:",n_tags

#words: 1691 #tags: 124


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [16]:
ctr = Counter(textfile.words)
print len(ctr)
print n_words

1691
1691


In [66]:
sentences = []
import codecs
txt = codecs.open("PoSTaggingCorpus/devel","r",encoding="utf-8")
sent = []
lines = txt.readlines()[2:]

In [92]:
txt = open("PoSTaggingCorpus/devel","r")
txt.readline()
txt.readline()
line = txt.readline()
sentences = []
sn = []
while line:
    if line == "\n":
       # print "LINEEND",sn
        sentences.append(sn)
        sn = []
        line = txt.readline()
    else: 
        #print "word"
        ln = line.split("\t")
        sn.append((ln[0],ln[1].strip("\n")))
        line = txt.readline()

print "#sentecses:",len(sentences) 
print sentences[0]

#sentecses: 147
[('Armonium', 'Smn'), ('.', 'FS')]


In [103]:
word2idx = {w: i + 1 for i, w in enumerate(words)}
print word2idx["casa"]

tag2idx = {t: i for i, t in enumerate(tags)}
print tag2idx["FS"]

729
57


In [109]:
max_len = 75
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)

# X is the value for the residual class
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["X"])


In [116]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

MemoryError: 